# Pythonic objects

## Object representation

In [5]:
import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return(bytes([ord(self.typecode)])+bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = (format(c, fmt_spec) for c in self)
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    

## classmethod vs staticmethod
- **classmethod**: defines a method that operates on the class and not on instances, it's first argument is the class not the object like it is for normal methods
- **staticmethod**: it has no argument referencing the object or the class, it is more like a plain function that happens to live inside of the class body

In [1]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

print(Demo.klassmeth())
print(Demo.klassmeth('spam'))
print(Demo.statmeth())
print(Demo.statmeth('spam'))


(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


## Formatted displays

In [7]:
print(format(Vector2d(1, 1), 'p'))
print(format(Vector2d(1, 1), '.3ep'))
print(format(Vector2d(1, 1), '0.5fp'))

<1.4142135623730951, 0.7853981633974483>
<1.414e+00, 7.854e-01>
<1.41421, 0.78540>


# Hashable vector2d
We need to implement the `__hash__` and `__eq__` methods and we need to make the object immutable. 

In [1]:
# Just the missing code from the previous vector2d class to make the class immutable

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in self(self.x, self.y))
    
v1 = Vector2d(1, 3)
v1.x = 3


AttributeError: property 'x' of 'Vector2d' object has no setter

### Hashing
The `__hash__` method should ideally return an integer which should take into account the hashes of the attributes that are used in the `__eq__` method, because immutable objects that compare equal should have the same hash.

In [ ]:
def __hash__(self):
    return hash(self.x) ^ hash(self.y)

## Private and protected attributes in python

## Saving space with `__slots__` class attribute
Python stores instance attributes in a per-instance dict named `__dict__` <br>
The `__slots__` class attribute can save a lot of memory by letting the interpreter store the instance attributes in a tuple instead of a dict. <br>
`__slots__` must be implemented in any individual class that wants to leverage it, it can not be inherited. <br>
If it implements slots, the instance won't be able to create any other attribute.

## Overriding class attributes